In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob, os, random
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from scipy.signal import savgol_filter
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
np.random.seed( 0 )
sns.set_style( "darkgrid" )


In [36]:
df = pd.read_parquet('C:/Users/lselig/selig-fa/finance/.data/fundamental_analysis_neglog.parquet')
df = df[(df.period == "Q2") & (df.calendarYear == "2022")]
print(df.columns)
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna(axis=0, inplace = True)
df = df.drop_duplicates()
targets = df.wk_curr_to_next_pct_inc.values

targets_binary = targets >= 1.0 # bools
targets_binary = np.array(targets_binary, dtype = "int")
plt.close()
plt.bar(targets_binary)
plt.show()


print(targets_binary)
tickers = df.symbol.values
periods = df.period.values
calendarYears = df.calendarYear.values
date = df.date.values
qualitative_data = np.array((tickers, periods, calendarYears, date)).T
df = df.drop(columns = ["period", "calendarYear", "symbol", "date", "open_next", "wk_curr_to_next_pct_inc"])
features = df.to_numpy()
print(targets.shape, features.shape)

Index(['symbol', 'calendarYear', 'period', 'revenue', 'costOfRevenue',
       'grossProfit', 'grossProfitRatio', 'researchAndDevelopmentExpenses',
       'generalAndAdministrativeExpenses', 'sellingAndMarketingExpenses',
       ...
       'netDebt', 'open_on', 'volume_on', 'vwap_on', 'open_next', 'open_prev',
       'volume_prev', 'vwap_prev', 'wk_curr_to_next_pct_inc',
       'wk_prev_to_curr_pct_inc'],
      dtype='object', length=115)


TypeError: bar() missing 1 required positional argument: 'height'

In [ ]:
scaler = StandardScaler()
features =  scaler.fit_transform(features)
features = np.clip(features, -5, 5)
print(features.shape)
n_features = features.shape[1]
print(n_features)
%matplotlib widget 
for i in range(n_features):
    print(i, np.nanmin(features[:, i]), np.nanmax(features[:, i]))
    sns.kdeplot(features[:, i])
plt.title("Distribution of final features")

In [ ]:
print(list(df))
print(features.shape, targets.shape)


In [ ]:
pca = PCA(n_components=2)
pca.fit(features.T)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
# print(pca.singular_values_)
# print(components)
# plt.hist2d(components[:, 0], components[:, 1], bins = 50)
components = pca.components_.T
print(components.shape)
# components[0,:]
plt.close()
plt.hist2d(components[:, 0], components[:, 1], bins = 60)
plt.show()
# plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation

clst = KMeans(n_clusters=3, random_state=0).fit(components)
# clst = AffinityPropagation( random_state=0).fit(components)
labels = clst.labels_.reshape(-1, 1)
pca_and_labels = np.hstack((components, labels))
print(pca_and_labels.shape)
clst.labels_.shape
fig, axs = plt.subplots(1, 2, figsize = (12, 8))
for g in np.unique(labels):
    idxs = np.where(pca_and_labels[:, 2] == g)[0]
    cluster = pca_and_labels[idxs]
    for i, c in enumerate(cluster):
        print(c[0], c[1], qualitative_data[:, 0][idxs[i]])
        axs[0].text(c[0], c[1], s = qualitative_data[:, 0][idxs[i]])
    axs[0].scatter(cluster[:, 0],  cluster[:, 1], label = g)
axs[0].set_xlabel("PCA1")
axs[0].set_ylabel("PCA2")
axs[1].hist2d(pca_and_labels[:, 0], pca_and_labels[:, 1], bins = 40)
axs[1].set_xlabel("PCA1")
axs[1].set_ylabel("PCA2")
axs[0].legend()
plt.show()

    
# plt.scatter(pca_and_labels[:, 0], pca_and_labels[: ,1], group = )
# plt.close()
# plt.hist(kmeans.labels_)
# plt.show()


In [ ]:
# the tickers where PCA2 is an outlier are all related to Federal National Mortgage Association (FNMA)
# not sure if it can be exploited
interesting = np.argwhere(pca_and_labels[: ,1] >= 0.08)
interesting_tickers = tickers[interesting]
# print(interesting_tickers)

interesting = np.argwhere(pca_and_labels[: ,1] <= -0.0365)
interesting_tickers = tickers[interesting]
print(interesting_tickers)

In [ ]:

interesting = np.argwhere(pca_and_labels[: ,0] >= 0.0055)
interesting_tickers = qualitative_data[interesting]
print(interesting_tickers)


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

plt.close()
# good = np.where(targets <= 100)[0]
corrs = [] 
for i in range(n_features):
    f = features[:, i]
    t = targets
    r = np.corrcoef(f, t)[0, 1]
    corrs.append(r)

sns.kdeplot(corrs)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print(mean_absolute_error(y_pred, y_train))
print(r2_score(y_pred, y_train))



NameError: name 'n_features' is not defined

In [26]:
df = pd.read_parquet('C:/Users/lselig/selig-fa/finance/.data/fundamental_analysis_neglog.parquet')
# df = pd.read_parquet('C:/Users/lselig/selig-fa/finance/.data/fundamental_analysis.parquet')
df = df[(df.period == "Q2") & (df.calendarYear == "2022")]
# df = df[(df.calendarYear == "2022")]
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna(axis=0, inplace = True)
df = df.drop_duplicates()
# targets = df.wk_curr_to_next_pct_inc.values
targets = df.open_next.values
# targets = df.open_prev.values
# targets = df.open_on.values
# targets_binary = targets >= 1.0 # bools
# targets_binary = np.array(targets_binary, dtype = "int")
tickers = df.symbol.values
periods = df.period.values
calendarYears = df.calendarYear.values
date = df.date.values
qualitative_data = np.array((tickers, periods, calendarYears, date)).T
# current_ratio = df.totalCurrentAssets / df.totalCurrentLiabilities
# df["share_number"] = df.commonStock + df.preferredStock
# market_cap = (df.share_number) * df.open_on
# enterprise_value = market_cap - df.cashAndCashEquivalents + df.totalDebt

# ratios = ["pe_ratio", "ps_ratio", "pcf_ratio", "pfcf_ratio",
#           "pb_ratio", "ev_sales", "ev_over_ebitda", "ev_to_cash_flow",
#           "earnings_yield", "free_cash_flow_yield", "debt_to_equity",
#           "debt_to_assets", "current_ratio"]
# price = df.open_prev
# df["pe_ratio"] = price / (df.netIncome_x / df.share_number)
# df["ps_ratio"] = price / (df.revenue / df.share_number)
# df["pcf_ratio"] = price / (df.operatingCashFlow / df.share_number)
# df["pfcf_ratio"] = market_cap / df.freeCashFlow
# df["pb_ratio"] = price  / (df.totalStockholdersEquity / df.share_number)
# df["ev_sales"] = enterprise_value / df.revenue
# df["ev_over_ebitda"] = enterprise_value / df.ebitda
# df["ev_to_cash_flow"] = enterprise_value / df.operatingCashFlow
# df["earnings_yield"] = (df.netIncome_x / df.share_number) / price
# df["free_cash_flow_yield"] = df.freeCashFlow / market_cap
# df["debt_to_equity"] = df.longTermDebt / df.totalStockholdersEquity
# df["debt_to_assets"] = df.longTermDebt / df.totalAssets
# df["current_ratio"] = df.totalCurrentAssets / df.totalCurrentLiabilities

# i = 7
# plt.close()
# sns.kdeplot(df[ratios[i]])
# plt.show()

df = df.drop(columns = ["vwap_on", "open_on", "period", "vwap_prev", "vwap_on", "open_prev",
                        "calendarYear", "symbol", "date", "open_next", "wk_curr_to_next_pct_inc",
                        "wk_prev_to_curr_pct_inc"])

for x in df.columns[-10:]:
    print(x)
features = df.to_numpy()
print(targets.shape, features.shape)

pcf_ratio
pfcf_ratio
pb_ratio
ev_sales
ev_over_ebitda
ev_to_cash_flow
earnings_yield
free_cash_flow_yield
debt_to_equity
debt_to_assets
(925,) (925, 118)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(mean_absolute_error(y_pred, y_test))
print(r2_score(y_test, y_pred))
print(y_pred)
for i in range(features.shape[1]):

    r = np.corrcoef(features[:, i], targets)[0, 1]
    print(np.round(r, 4))
    

(740, 118) (185, 118) (740,) (185,)
71.57303782956774
-0.03773341239502681
[ 4.39753186e+00  1.12736488e+02 -1.92560722e+01  2.11268763e+01
 -2.77261291e+01  1.62848815e+02  6.34057508e+00 -5.66512698e+01
  4.68287012e+01  9.60371254e+01 -5.17398818e+01  4.09287956e+01
 -2.32789942e+01  1.87427548e+02  4.23603105e+01  5.23770631e+01
  1.03963653e+02  6.71946657e+01  1.31519391e+01  1.79363200e+01
  1.33469199e+02  2.20931924e+02  5.74223132e+01 -7.41579299e+01
  1.15276329e+02  1.30733338e+02 -5.55264466e+00  2.52063150e+01
  1.43579544e+02  1.33969153e+02  1.57951405e+02 -1.82381563e+01
  7.28114423e+01 -3.78925773e+01  1.30633235e+01  1.57139848e+02
  1.45772028e+01  1.03112527e+02  3.14658256e+01 -9.64642384e-01
  2.59759799e+02  1.36324571e+02  1.39966132e+02  6.97490371e+01
 -6.94392850e-01  2.29806923e+02  9.96147686e+01  3.10921934e+02
  7.62778487e+01  7.53455688e+01  2.16012665e+02  7.44520760e+01
  1.01410395e+02  1.45093605e+02  6.87057696e+01  9.26246614e+01
 -2.78156410e+0

In [34]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
print(targets_binary)
X_train, X_test, y_train, y_test = train_test_split(features, targets_binary, test_size = 0.2, random_state = 42)
print(y_train)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
print(y_pred)
print(classification_report(y_pred, y_train))

[1 1 0 ... 1 1 0]
[0 1 0 ... 0 0 1]
[1 1 1 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.11      1.00      0.19        45
           1       1.00      0.66      0.79      1087

    accuracy                           0.67      1132
   macro avg       0.55      0.83      0.49      1132
weighted avg       0.96      0.67      0.77      1132

